In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [ ]:
all_matches = []
standings_url = "https://www.basketball-reference.com/leagues/NBA_2024.html#all_per_game_team-opponent"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)

standings_table_east = soup.select('table.stats_table')[0]
standings_table_west = soup.select('table.stats_table')[1]
links_east = standings_table_east.find_all('a')
links_west = standings_table_west.find_all('a')
links = links_east + links_west
links = [l.get("href") for l in links]
team_urls = [f"https://www.basketball-reference.com{l}" for l in links]

previous_season = soup.select("a.prev")[0].get("href")
standings_url = f"https://www.basketball-reference.com{previous_season}"

for team_url in team_urls:
  team_name = team_url.split("/")[-2]
  data = requests.get(team_url)
  soup = BeautifulSoup(data.text)
  results = soup.find_all('li', class_='result')
  games_data = []
  for result in results:
    games_data.append(result.find('span', class_='poptip')['tip'])

  processed_data = [re.sub(r'^\d+\.\s*', '', item) for item in games_data]
  team_data = pd.DataFrame([item.split(', ') for item in processed_data], columns=['Date', 'Result', 'Score'])
  team_data['Home'] = team_data['Date'].str.contains('@')
  team_data['Date'] = team_data['Date'].str.replace(r'@\s*\w+', '', regex=True)
  team_data['Opponent'] = team_data['Result'].str.extract(r'(\w+)$')
  team_data['Result'] = team_data['Result'].str.replace(r'\s*\w+$', '', regex=True)
  team_data['Record'] = team_data['Result'].str.extract(r'\((\d+-\d+)\)')
  team_data['Result'] = team_data['Result'].apply(lambda x: 'W' if 'beat' in x else 'L')
  team_data['Team'] = team_name

  all_matches.append(team_data)
  time.sleep(1)

IndexError: list index out of range

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.to_csv("matches.csv")